In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import h3pandas

In [2]:
input_dir = Path('combined_h3')
output_dir = Path.cwd()

# Get a list of all GPKG files in the folder
d_files = list(sorted(input_dir.glob('discrete*.parquet')))
c_files = list(sorted(input_dir.glob('continuous*.parquet')))

In [3]:
combined_list = d_files + c_files
combined_list = sorted(combined_list, key=lambda x: int(x.stem.split('-')[-1]))

In [4]:
num_files_to_open = 1000  #Layers to join

In [5]:
%%time
for i, file in enumerate(combined_list[:num_files_to_open]):
        col_name = file.stem  # Extracting the column name from the file name
        df = pd.read_parquet(file).drop(columns=['geometry'])
    
        #This is classifying the continuous raster into 10 quantile classes
        if (i % 2) == 0:
            pass
        else:
            df['Value'] = pd.qcut(df['Value'], q=10, labels=False)
            
        df.columns = [col_name, *df.columns[1:]]
        if i == 0:
            combined_df = df.copy()
        else:
            combined_df = combined_df.join(df)
        

CPU times: total: 1.75 s
Wall time: 12 s


In [6]:
h3_df=combined_df.h3.h3_to_geo_boundary() 

In [7]:
h3_df.to_file('raster_combined.gpkg', driver='GPKG', mode='w')